# Object Tracking

In [1]:
import cv2 
import numpy as np

In [2]:
# set some parameter values using Shi-Tomasi Corner Detection
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize =7)


In [4]:
# smaller window size are more prone to noise, larger windown
# size may not be sensetive to smaller motions of the points, points
# may be moving but it feels the frame is not moving
# maxLevel creates image pyramid with different resolution
# next we use pip operator providing two criteria to perform Lucas-Kanade optical Flow
# provide max iterations and provide epsilon which is 0.03. More iteration
# means more exhaustive serach of points, smaller eps means gonna finish earlier
# esp exchange speed of your tracking vs accuracy of your tracking
lk_params = dict(winSize = (200, 200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [ ]:
cap =cv2.VideoCapture(0)
# reading very first frame
ret, prev_frame = cap.read()
# converting fisrt frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# points to track using good feature trackng
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)
mask = np.zeros_like(prev_frame)


while True:
    ret, frame = cp.read()

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)

    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]

    for i, (new, prev) in enumerate(zip(good_new, good_prev)):

        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()

        mask = cv2.line(mask, (x_new, y_new),(x_prev, y_prev), (0, 255, 0), 3)

        frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)

    img = cv2.add(frame, mask)
    cv2.imshow('Tracking', img)

    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()





